## Importing Libraries

In [1]:
import pandas as pd
import folium
import numpy as np
import geopandas as gpd
from pandas import DataFrame
import json

## Reading files

In [ ]:
# Assign spreadsheet filename to `file`
file = './Data/CVRPStats.xlsx'
file2 = './Data/ces3results.xlsx'
file3 = './Data/income.xlsx'

# Load spreadsheet
xl = pd.ExcelFile(file)
xl2 = pd.ExcelFile(file2)
xl3 = pd.ExcelFile(file3)

# Load a sheet into a DataFrame by name: df1
cvrpData = xl.parse('CVRP')
cesData = xl2.parse('CES')
incomeData = xl3.parse('Sheet1')

In [ ]:
cvrpData.head()

In [ ]:
cesData.head()

In [ ]:
incomeData.head()

## Data Parsing

In [ ]:
def datacleaner(data, filer):
    df = DataFrame(data, columns= ['zipcode','Size_of_adjusted_gross_income',filer])
    Income = df.loc[df['Size_of_adjusted_gross_income'].isin(['$1 under $25,000', '$25,000 under $50,000', '$50,000 under $75,000', '$75,000 under $100,000', '$100,000 under $200,000'])]
    Income = Income.drop(['Size_of_adjusted_gross_income'], axis=1)
    Income['zipcode'] = Income['zipcode'].apply(np.int)
    Income.dropna()
    Income['zipcode'] = Income['zipcode'].astype(np.str)
    
    def number(row):
        if row == ".":
            return 0
        else:
            return int(row)
    Income[filer] = Income[filer].apply(lambda label: number(label))
    Income[filer] = Income.groupby(['zipcode'])[filer].transform('sum')
    Income[filer] = Income[filer].astype(np.int)
    Income.loc[Income.zipcode == '99999', filer] = 0
    Income = Income.drop_duplicates(subset='zipcode', keep="first")
    return Income

## Map Visualization

In [ ]:
def mapMaker(filer, title, data):
    map = folium.Map(location=(37.8272, -122.2913), zoom_start=6)
    folium.Choropleth(geo_data='./Data/stanford-dc841dq9031-geojson.json',
               data=data,
               columns=['zipcode', filer],
               key_on='feature.properties.zcta',
               fill_color= 'BuPu', fill_opacity=1.0, line_opacity=0.2,
               legend_name=title).add_to(map)
    return map

## Different Incomes

In [ ]:
singleIncome = datacleaner(incomeData, 'Number of single returns')
singleMap = mapMaker('Number of single returns','Single Income Returns', singleIncome)
singleMap

In [ ]:
jointIncome = datacleaner(incomeData, 'Number of joint returns')
jointMap = mapMaker('Number of joint returns','Joint Income Returns', jointIncome)
jointMap